In [1]:
import numpy as np
import random

### Генератор Лемера

In [ ]:
class Linear_Low:
    a = 2**16 + 1
    c = 119
    x0 = 1  # тюряга #

    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=np.uint32)
        seq[0] = self.x0

        for i in range(0, n - 1):
            seq[i+1] = (self.a*seq[i] + self.c) 

        seq = np.array([seq % (2**8)], dtype=np.uint8)

        return seq

class Linear_High:
    a = 2**16 + 1
    c = 119
    x0 = 1  # тюряга #

    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=np.uint32)
        seq[0] = self.x0

        for i in range(0, n - 1):
            seq[i+1] = (self.a*seq[i] + self.c)

        seq = np.array([seq >> 24], dtype=np.uint8)

        return seq
        
            

In [ ]:
de1 = Linear_Low()
de2 = Linear_High()

print(de1.generate_bytes(20))
print(de2.generate_bytes(20))

### Генератор L20

In [ ]:
class L20:
    def __init__(self, x_init: np.array):
        self.x_init = x_init

    def generate_bits(self, n: int):
        seq = np.concatenate([np.array(self.x_init, dtype=np.uint8), np.zeros(n - 20, dtype=np.uint8)])

        for i in range(20, n):
            seq[i] = seq[i - 3] ^ seq[i - 5] ^ seq[i - 9] ^ seq[i - 20]

        seq = seq % 2

        return seq
            

In [ ]:
smp = np.array([random.randint(0, 1) for _ in range(20)])
print(smp)
de_L20 = L20(smp)

print(de_L20.generate_bits(100))

### Генератор L89

In [ ]:
class L89:
    def __init__(self, x_init: np.array):
        self.x_init = x_init

    def generate_bits(self, n: int):
        seq = np.concatenate([np.array(self.x_init, dtype=np.uint8), np.zeros(n - 89, dtype=np.uint8)])

        for i in range(89, n):
            seq[i] = seq[i - 38] ^ seq[i - 89]

        seq = seq % 2

        return seq
            

In [ ]:
smp = np.array([random.randint(0, 1) for _ in range(89)])
de_L89 = L89(smp)

print(de_L89.generate_bits(90))

### Генератор Вольфрама

In [ ]:
# В ПІТОНІ НЕМА ВБУДОВАНОГО ЦИКЛІЧНОГО ЗСУВУ
def rcs(n: np.uint32, rotations):
    return (n >> rotations | n << (32-rotations))

def lcs(n: np.uint32, rotations):
    return (n << rotations | n >> (32-rotations))

class Wolfram:
    def __init__(self, r0: np.uint32):
        self.r0 = r0

    def generate_bits(self, n: int):
        r_i = self.r0
        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = r_i % 2
            r_i = lcs(r_i, 1) ^ (r_i | rcs(r_i, 1))  

        return seq
            

In [ ]:
de_wolfram = Wolfram(1)

print(de_wolfram.generate_bits(100))

### Генератор BM

In [8]:
class BM:
    def __init__(self, p, a):
        self.p = p
        self.a = a

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(0, self.p - 1) 

        for i in range(1, n):
            seq[i] = pow(self.a, seq[i - 1], self.p)

        seq = np.array([seq < (self.p - 1) / 2], dtype=np.uint8) 

        return seq
    
    #todo
    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(0, self.p - 1) 

        for i in range(1, n):
            seq[i] = pow(self.a, seq[i - 1], self.p)

        seq = np.array([seq < (self.p - 1) / 2], dtype=np.uint8) 

        return seq
            

In [9]:
p = int("CEA42B987C44FA642D80AD9F51F10457690DEF10C83D0BC1BCEE12FC3B6093E3", 16)
a = int("5B88C41246790891C095E2878880342E88C79974303BD0400B090FE38A688356", 16)

de_BM = BM(p, a)

print(de_BM.generate_bits(1000))

[[1 0 1 0 0 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 0
  0 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 1 0 1 1
  1 1 0 0 1 0 0 1 1 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1
  0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 1 1 1 0
  1 1 0 0 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0 0 1 0 0 0 0 1 1 1 0 0 1 1 1 1 1 0
  0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0
  0 1 1 1 1 0 0 1 1 1 0 1 0 1 1 0 1 0 1 1 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1
  0 1 1 1 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 1 0 1 0
  0 0 0 1 1 0 0 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 0 0 1 0 1 0 1 0 1 0 0 1
  0 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 1 1 1 1
  1 0 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 0 1 0 0 1
  1 1 0 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 0 1 1 0 0 0 0 1 0
  0 1 0 0 1 0 0 1 1 0 0 1 0 0 1 1 1 1 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 0
  1 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1 0 1 

### Генератор BBS

In [10]:
class BBS:
    def __init__(self, p, q):
        self.n = p*q

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(2, self.n - 1) 

        for i in range(1, n):
            seq[i] = pow(seq[i - 1], 2, self.n)

        seq = np.array([seq % 2], dtype=np.uint8) 

        return seq
    
    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(2, self.n - 1) 

        for i in range(1, n):
            seq[i] = pow(seq[i - 1], 2, self.n)

        seq = np.array([seq % (2**8)], dtype=np.uint8) 

        return seq
            

In [12]:
p = int("D5BBB96D30086EC484EBA3D7F9CAEB07", 16)
q = int("425D2B9BFDB25B9CF6C416CC6E37B59C1F", 16)

de_BBS = BBS(p, q)

print(de_BBS.generate_bytes(1000))

[[135 162 192 178 195  46 176  65 192  43 129 213  98  87 186 154 105 171
   42  38 237  89  69 214 176 163  21 241 205 106 127 233 175 142 105 199
  254 227  33  30  29 221 179 164 185 141  90 213 120  64  33 188 128 236
   15  29  27  59  48  47 252 117  97 238 219 193  86 217  45 154  91  80
   31  55 156 111 136   4 131 180  94 122 207  71  84 179  98  46  66 160
  235 204 246 166  42 159 151 121 205 132  35 106  64 223 111 107  58 216
  215 208  22 233 113 118 247 171 151 255 175  69  75 222  34  18 192  31
   26  73 230 211 218 121 215  54 231 101 178  92 234 158 101  14  10  18
  250 241  44 123 123  57 249 224  55   0 244 124 115  80 106  91 232 222
   46 150 152  10 243  12 203 178 191   4  57  25 246 225 130 102 253  35
  113 171 158  70 223   0 248  89 142   1 198  65 144  31 101 217  33  53
  245 204 229  31 124 249  63 119   9  58  41 219 120 196 123 209 125  82
  246 150 195 206 202 150 176 159 130 198  89  22 198  53  93  37 105 222
  241  23  64  40  86  93 112 154  64 

#### Криптографічно нестійкий генератор

#### Криптографічно стійкий генератор